<!-- @format -->

# 1. neo4j Quick Start

- 構造化データを Knowledge Graph を構築
- データベースは neo4j


In [ ]:
import pandas as pd
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

In [ ]:
load_dotenv("../.env")

<!-- @format -->

## Sample data : Movie Lens

https://github.com/tomasonjo/blog-datasets/tree/main/movies


In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv",
)
df

<!-- @format -->

## Build Knowledge Graph


In [ ]:
graph = Neo4jGraph()

In [ ]:
movies_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [ ]:
# build query
graph.query(movies_query)

<!-- @format -->

## Query sample


<!-- @format -->

#### 全部の映画のノード


In [ ]:
# all Movie nodes
query = "MATCH (m:Movie) RETURN m"
result = graph.query(query)
print(result)

<!-- @format -->

### Tom Hanks の出演映画


In [ ]:
query = "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN m"
result = graph.query(query)

print(result)

<!-- @format -->

### Toy Story にエッジのあるすべてのノード


In [ ]:
query = """MATCH (m:Movie {title: "Toy Story"})
OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Person)
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)
OPTIONAL MATCH (m)-[:IN_GENRE]->(g:Genre)
RETURN m, d, a, g"""
result = graph.query(query)

print(result)

<!-- @format -->

### 1996 年公開の映画で評価の高い上位 5 つ


In [ ]:
query = """MATCH (m:Movie)
WHERE m.released.year = 1996
RETURN m.title, m.imdbRating
ORDER BY m.imdbRating DESC
LIMIT 5"""

result = graph.query(query)
print(result)

In [ ]:
graph.refresh_schema()
print(graph.schema)